# <div align="center" style="color: #ff5733;">New Pos Addition and Mapping</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\pos_add_20240823_file1.csv")
df.shape

(5, 8)

In [3]:
# df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data May042024.xlsx", sheet_name = 'Sheet1')
# df.head(10)

In [4]:
df.head()

,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM,Brand
0,Event POS Faith Glorious Fit Sales Center Kiko 2,Standalone,For Opening,"August 23, 2024",MK63TQM,North Luzon,Sander Maghirang,Faith Glorious Fit
1,Event POS Faith Glorious Fit Sales Center Navotas,Standalone,For Opening,"August 23, 2024",M7UX250,North Luzon,Sander Maghirang,Faith Glorious Fit
2,Event POS Faith Glorious Fit Sales Center Conc...,Standalone,For Opening,"August 23, 2024",MZBG4JP,North Luzon,Sander Maghirang,Faith Glorious Fit
3,Event POS Faith Glorious Fit Sales Center Kiko 1,Standalone,For Opening,"August 23, 2024",MCW3A7O,North Luzon,Sander Maghirang,Faith Glorious Fit
4,Event POS Faith Glorious Fit Sales Center Las ...,Standalone,For Opening,"August 23, 2024",MYQ92OV,South Luzon,Grace Belarmino,Faith Glorious Fit


In [5]:
df.rename(columns = {'Brand': 'BrandStores'}, inplace = True)
df.columns

Index(['POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY',
       'REGION', 'TSM', 'BrandStores'],
      dtype='object')

In [6]:
df['BrandStores'].value_counts()

BrandStores
Faith Glorious Fit    5
Name: count, dtype: int64

Check whether we have duplicate purple key in the data provided by the SIL team

In [7]:
df['PURPLE KEY'].value_counts()

PURPLE KEY
MK63TQM    1
M7UX250    1
MZBG4JP    1
MCW3A7O    1
MYQ92OV    1
Name: count, dtype: int64

In [8]:
df = df[['BrandStores', 'POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY', 'REGION', 'TSM']]
df

,BrandStores,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM
0,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Kiko 2,Standalone,For Opening,"August 23, 2024",MK63TQM,North Luzon,Sander Maghirang
1,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Navotas,Standalone,For Opening,"August 23, 2024",M7UX250,North Luzon,Sander Maghirang
2,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Conc...,Standalone,For Opening,"August 23, 2024",MZBG4JP,North Luzon,Sander Maghirang
3,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Kiko 1,Standalone,For Opening,"August 23, 2024",MCW3A7O,North Luzon,Sander Maghirang
4,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Las ...,Standalone,For Opening,"August 23, 2024",MYQ92OV,South Luzon,Grace Belarmino


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BrandStores      5 non-null      object
 1   POS NAME         5 non-null      object
 2   STORE LOCATION   5 non-null      object
 3   STATUS           5 non-null      object
 4   DATE ACTIVATION  5 non-null      object
 5   PURPLE KEY       5 non-null      object
 6   REGION           5 non-null      object
 7   TSM              5 non-null      object
dtypes: object(8)
memory usage: 448.0+ bytes


In [10]:
df.rename(columns={'POS NAME':'POS', 'DATE ACTIVATION':'DateOpened', 'STORE LOCATION': 'StoreLocation', 'STATUS':'Status', 'PURPLE KEY':'PurpleKey', 'REGION':'Region'}, inplace = True)

In [11]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Kiko 2,Standalone,For Opening,"August 23, 2024",MK63TQM,North Luzon,Sander Maghirang
1,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Navotas,Standalone,For Opening,"August 23, 2024",M7UX250,North Luzon,Sander Maghirang
2,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Conc...,Standalone,For Opening,"August 23, 2024",MZBG4JP,North Luzon,Sander Maghirang
3,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Kiko 1,Standalone,For Opening,"August 23, 2024",MCW3A7O,North Luzon,Sander Maghirang
4,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Las ...,Standalone,For Opening,"August 23, 2024",MYQ92OV,South Luzon,Grace Belarmino


In [12]:
from datetime import datetime
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%m-%Y')
# Format the datetime object to YYYY-MM-DD
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')

ValueError: time data "August 23, 2024" doesn't match format "%d-%m-%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [13]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%B %d, %Y')


In [14]:
print(df['DateOpened'].dtype)
# datetime64[ns]
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

datetime64[ns]
2024-08-23 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BrandStores    5 non-null      object
 1   POS            5 non-null      object
 2   StoreLocation  5 non-null      object
 3   Status         5 non-null      object
 4   DateOpened     5 non-null      object
 5   PurpleKey      5 non-null      object
 6   Region         5 non-null      object
 7   TSM            5 non-null      object
dtypes: object(8)
memory usage: 448.0+ bytes


In [15]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%b-%y')

ValueError: time data "2024-08-23" doesn't match format "%d-%b-%y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [16]:
print(df['DateOpened'].dtype)
# datetime64[ns]

object


In [17]:
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00

2024-08-23


In [18]:
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

AttributeError: Can only use .dt accessor with datetimelike values

In [19]:
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%Y-%m-%d')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   BrandStores    5 non-null      object        
 1   POS            5 non-null      object        
 2   StoreLocation  5 non-null      object        
 3   Status         5 non-null      object        
 4   DateOpened     5 non-null      datetime64[ns]
 5   PurpleKey      5 non-null      object        
 6   Region         5 non-null      object        
 7   TSM            5 non-null      object        
dtypes: datetime64[ns](1), object(7)
memory usage: 448.0+ bytes


In [21]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Kiko 2,Standalone,For Opening,2024-08-23,MK63TQM,North Luzon,Sander Maghirang
1,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Navotas,Standalone,For Opening,2024-08-23,M7UX250,North Luzon,Sander Maghirang
2,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Conc...,Standalone,For Opening,2024-08-23,MZBG4JP,North Luzon,Sander Maghirang
3,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Kiko 1,Standalone,For Opening,2024-08-23,MCW3A7O,North Luzon,Sander Maghirang
4,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Las ...,Standalone,For Opening,2024-08-23,MYQ92OV,South Luzon,Grace Belarmino


In [22]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
try:
    d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
    print("Table still exists")
except:
    print("Table dropped")

Query is running:   0%|          |Table dropped


In [23]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1
as 
select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;
"""
client.query(sq)



QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a76546af-5d96-4b91-8192-e458e3d209e2>

In [24]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of d is:\t{d.shape}")

Job ID 96102647-2ec8-40fb-a20e-2da9abbea428 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of d is:	(887, 8)


In [25]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""

dfsil = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 9e796d3e-880f-4adb-b616-f211e990ed72 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [26]:
dfsil.BrandStores.value_counts().sort_index()

BrandStores
Affordables                     2
Ansons                         15
ArkHome                         4
Automatic Centre               11
Chrisrence Home Appliances      2
Dizon Furniture                 1
ESLA                           51
FC Home                       109
Faith Glorious                  9
Faith Glorious Fit              3
Gadgetmate                     10
Hardware Sugar                  2
Hazrat                          6
Home Along                     96
Home Factory                    2
Homeworks                      15
JSL                             7
JTG                             1
Jhunpyo                        12
Junliz                          8
KUM Home Center                 3
Kservico                       10
MemoXpress                     54
PHPS                            2
Power Mac                       5
Power Mac Center                5
Presnet                        21
Prince Retail                  11
Prince and Aviel                3
RC

In [27]:
dfsil.shape

(887, 8)

In [28]:
dfsil.columns

Index(['BrandStores', 'POS', 'StoreLocation', 'Status', 'DateOpened',
       'PurpleKey', 'Region', 'TSM'],
      dtype='object')

In [29]:
dfsil[dfsil['PurpleKey'] =='MA9VNCJ']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
594,Rulls,Rulls Prince Hypermarket Camotes,Standalone,POS Operational,2024-08-18,MA9VNCJ,VISAYAS,Rona Jay Sinangote


In [30]:
# Set table_id to the ID of the table to append to.
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  

job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 892 rows and 8 columns to prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1


In [31]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is: {df.shape}")

Job ID b8c31ac3-4488-49e8-aca5-3a4221e2d5ec successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is: (892, 8)


In [32]:
df.tail(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
882,FC Home,FC Home Center Siniloan,Standalone,POS Operational,2023-09-15,MANB6XQ,South Luzon 2,Christian Klide Alberto
883,Workstation,Workstation Home Appliances Pavilion Mall 1,Pavillon Mall,POS Operational,2023-10-13,MN8MWPJ,South Luzon 2,Christian Klide Alberto
884,Workstation,Workstation Home Appliances Target Mall,Target Mall,POS Operational,2023-10-13,M8I4QGT,South Luzon 2,Christian Klide Alberto
885,Workstation,Workstation Home Appliances Central Mall,Central Mall,POS Operational,2023-10-13,MTNCOR5,South Luzon 2,Christian Klide Alberto
886,Xtreme,Xtreme Appliances Calamba,Carmel Mall,POS Operational,2023-10-15,MUBPW04,South Luzon 2,Christian Klide Alberto
887,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Kiko 2,Standalone,For Opening,2024-08-23,MK63TQM,North Luzon,Sander Maghirang
888,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Navotas,Standalone,For Opening,2024-08-23,M7UX250,North Luzon,Sander Maghirang
889,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Conc...,Standalone,For Opening,2024-08-23,MZBG4JP,North Luzon,Sander Maghirang
890,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Kiko 1,Standalone,For Opening,2024-08-23,MCW3A7O,North Luzon,Sander Maghirang
891,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Las ...,Standalone,For Opening,2024-08-23,MYQ92OV,South Luzon,Grace Belarmino


In [33]:
df['BrandStores'].value_counts().sort_index()

BrandStores
Affordables                     2
Ansons                         15
ArkHome                         4
Automatic Centre               11
Chrisrence Home Appliances      2
Dizon Furniture                 1
ESLA                           51
FC Home                       109
Faith Glorious                  9
Faith Glorious Fit              8
Gadgetmate                     10
Hardware Sugar                  2
Hazrat                          6
Home Along                     96
Home Factory                    2
Homeworks                      15
JSL                             7
JTG                             1
Jhunpyo                        12
Junliz                          8
KUM Home Center                 3
Kservico                       10
MemoXpress                     54
PHPS                            2
Power Mac                       5
Power Mac Center                5
Presnet                        21
Prince Retail                  11
Prince and Aviel                3
RC

In [34]:
# Find duplicate values in 'POS' column
duplicates = df[df.duplicated('POS', keep=False)]

# Print the entire rows for duplicate entries in 'POS'
if not duplicates.empty:
    print("Duplicate values in 'POS' column:")
    duplicates.to_csv("Temp.csv")
else:
    print("No duplicate values found in 'POS' column.")
    
dd = pd.read_csv("Temp.csv")
dd.sort_values(by='POS')

Duplicate values in 'POS' column:


,Unnamed: 0,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,6,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,MV4UO0X,NaN,NaN
1,7,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,M9ZK6IU,NaN,NaN
29,850,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MGP2AQI,NCR North,Ben Antonio Lacsamana
28,848,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MN3GL66,NCR North 2,Ben Antonio Lacsamana
30,856,Home Along,Home Along Malanday,Standalone,POS Operational,2022-07-11,MNBW4RZ,NCR North,Ben Antonio Lacsamana
31,857,Home Along,Home Along Malanday,NaN,NaN,2022-07-11,MURFAIV,NCR North,Ben Antonio Lacsamana
19,769,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MU82X9Z,NCR South,Klein Justine Tamayo
20,770,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MDGYHQJ,NCR South,Klein Justine Tamayo
24,785,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MDO3KI4,NCR South,Klein Justine Tamayo
22,783,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MRGP0WQ,NCR South,Klein Justine Tamayo


In [35]:
pd.set_option('display.max_rows', None)
# Count the occurrences of each value in the 'PurpleKey' column
purple_key_counts = df['PurpleKey'].value_counts()

# Filter to include only values with count >= 2
filtered_counts = purple_key_counts[purple_key_counts >= 2]

# Display the filtered counts
print(filtered_counts)


Series([], Name: count, dtype: int64)


In [36]:
df[df['PurpleKey']== 'MK63TQM']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
887,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Kiko 2,Standalone,For Opening,2024-08-23,MK63TQM,North Luzon,Sander Maghirang


In [37]:
sq = """drop table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=7030ee8e-e803-47e1-b6d2-ca166120aa98>

In [38]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING as select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=62fc50fc-431d-41cf-86b9-9efa6463a9a9>

In [39]:
client.query("commit")

QueryJob<project=prj-prod-dataplatform, location=US, id=40cd8218-a350-4309-a7ff-46b8e95b3fb0>

In [40]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.shape

Job ID f36c1802-b53e-4dbf-a1b6-32d1e1b7ba20 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(892, 8)

In [41]:
df[df['BrandStores']=='VMax Mobile'].sort_values(by='DateOpened')

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
517,VMax Mobile,Lets Mobile Phone and accessories Store,Divimall Cabuyao,POS Operational,2024-07-27,MSNZ3XL,South Luzon,Aubrey Lalong Isip
518,VMax Mobile,Delle General Merch Harmony Mall,Harmony Village Mall,POS Operational,2024-07-27,M3YOW2R,South Luzon,Aubrey Lalong Isip
519,VMax Mobile,Vmax Waltermart Sta Rosa,Waltermart Starosa,POS Operational,2024-07-27,MJ0WZAK,South Luzon,Aubrey Lalong Isip
520,VMax Mobile,Vmax Central Balibago,Victory Central Mall Balibago,POS Operational,2024-07-27,MWN8FVI,South Luzon,Aubrey Lalong Isip
521,VMax Mobile,Vmax Central Binan 2,Central Mall Binan,POS Operational,2024-07-27,ME4NBT7,South Luzon,Aubrey Lalong Isip
522,VMax Mobile,Vmax Central Binan,Central Mall Binan,POS Operational,2024-07-27,MZEO85E,South Luzon,Aubrey Lalong Isip
523,VMax Mobile,Realme Target Mall Balibago,Target Mall Balibago,For Opening,2024-08-02,MXACV0S,South Luzon,Aubrey Lalong Isip
524,VMax Mobile,Vmax Waltermart Cabuyao,Waltermart Cabuyao,For Opening,2024-08-02,MKPNC4B,South Luzon,Aubrey Lalong Isip
525,VMax Mobile,St Vincent Telecom Target Mall,Target Mall Starosa,For Opening,2024-08-02,MVTZGDH,South Luzon,Aubrey Lalong Isip
526,VMax Mobile,Delle General Merch Target Mall,Target Mall Balibago,For Opening,2024-08-02,MLX9FJP,South Luzon,Aubrey Lalong Isip


In [42]:
df.head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Balibago,LOT 5 BLK 9 GARNET ST. RHINELAND COMMERCIAL CO...,Operational,2023-08-01,MOG243Y,South Luzon 2,Klide Alberto
1,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Pasig,Standalone,POS Operational,2024-08-02,MOJ34ME,ENCR,Martin Lin
2,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center San ...,Standalone,POS Operational,2024-08-02,MB3YSES,CAMANAVA,Paul Palonpon
3,Prince Retail,Prince Lopez,Standalone,For Opening,2024-08-15,MNRVWJ1,Bicol,Penfrancia Ayo
4,Home Along,Home Along Talavera,Standalone,POS Operational,2024-08-15,MZKJOGD,North Luzon,Rhea Geronimo
5,Xtreme,Xtreme Appliances Anabu,None,None,2022-08-27,MR2YDE8,None,None
6,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,M9ZK6IU,None,None
7,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,MV4UO0X,None,None
8,Home Along,Home Along Sumulong,None,None,2022-07-12,M2G78K4,None,None
9,Dizon Furniture,Dizon Furniture Tiendesitas,None,None,2022-07-23,MOE3JQH,None,None


In [43]:
df.sort_values('DateOpened', ascending=False)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
274,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Kiko 1,Standalone,For Opening,2024-08-23,MCW3A7O,North Luzon,Sander Maghirang
600,Rulls,Rulls Kiosk Prince Sogod Cebu,Standalone,For Opening,2024-08-23,M6QAVYH,VISAYAS,Rona Jay Sinangote
172,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Las ...,Standalone,For Opening,2024-08-23,MYQ92OV,South Luzon,Grace Belarmino
272,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Navotas,Standalone,For Opening,2024-08-23,M7UX250,North Luzon,Sander Maghirang
273,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Kiko 2,Standalone,For Opening,2024-08-23,MK63TQM,North Luzon,Sander Maghirang
275,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Conc...,Standalone,For Opening,2024-08-23,MZBG4JP,North Luzon,Sander Maghirang
599,Rulls,Rulls Prince Hypermarket Camotes,Standalone,POS Operational,2024-08-18,MA9VNCJ,VISAYAS,Rona Jay Sinangote
793,Power Mac,Power Mac Center Greenbelt,Ayala Mall Circuit,For Opening,2024-08-16,M9T2W5I,NCR South 1,Klein Justine Tamayo
170,Power Mac,Power Mac Center Festival Supermall,Festival Supermall,For Opening,2024-08-16,MUVWA6D,NCR South 2,Grace Belarmino
171,Power Mac,Power Mac Center Manila Bay,Ayala Mall Manila Bay,For Opening,2024-08-16,MBJZIEO,NCR South 2,Grace Belarmino


In [44]:
df[df['PurpleKey'] =='MK63TQM']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
273,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Kiko 2,Standalone,For Opening,2024-08-23,MK63TQM,North Luzon,Sander Maghirang


In [45]:
df.groupby(['BrandStores', 'DateOpened'])['PurpleKey'].nunique()

BrandStores                 DateOpened
Affordables                 2024-01-22      1
                            2024-04-12      1
Ansons                      2024-04-15     15
ArkHome                     2024-06-07      4
Automatic Centre            2023-07-15      6
                            2023-07-16      1
                            2023-07-22      4
Chrisrence Home Appliances  2024-07-13      2
Dizon Furniture             2022-07-23      1
ESLA                        2024-02-15     30
                            2024-04-22     21
FC Home                     2023-08-01     48
                            2023-09-01     20
                            2023-09-05      1
                            2023-09-14     25
                            2023-09-15     11
                            2023-11-10      1
                            2024-01-15      1
                            2024-03-26      1
                            2024-04-24      1
Faith Glorious              2024-01-15   

In [46]:
df[df['BrandStores']=='Rulls']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
11,Rulls,Rulls Sta Lucia,Sta Lucia Mall,For Opening,2024-04-15,M58WV1L,NCR East,Albee David
13,Rulls,MBC Gaisano Inline Binangonan,Gaisano Inline Binangonan,For Opening,2024-04-15,M38S4QA,NCR East,Albee David
14,Rulls,Rulls Antipolo,Robinsons Antipolo,For Opening,2024-04-15,M8U05KZ,NCR East,Albee David
15,Rulls,Audionet Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,M8BPO3Y,NCR East,Albee David
16,Rulls,MBC Montalban Town Center,Montalban Town Center,For Opening,2024-04-15,MI9KLNA,NCR East,Albee David
17,Rulls,Fun Mobile Ayala Marikina,Ayala Malls Marikina,For Opening,2024-04-15,MK4CZP2,NCR East,Albee David
18,Rulls,Realme Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,MBLR71F,NCR East,Albee David
22,Rulls,Oppo Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,MTEHX0R,NCR East,Albee David
23,Rulls,Mobilecraze K3 PG Marikina,Puregold Marikina,For Opening,2024-04-15,MW6HOE7,NCR East,Albee David
25,Rulls,Rulls Unimart Capitol,Unimart Capitol Common,For Opening,2024-04-15,MNGI12Q,NCR East,Albee David


In [47]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data.xlsx", index = False)

In [48]:
df[df['PurpleKey']=='MK63TQM']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
273,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Kiko 2,Standalone,For Opening,2024-08-23,MK63TQM,North Luzon,Sander Maghirang
